In [2]:
# Import packages
import cv2 
import mediapipe as mp
import numpy as np
import time



#Build Keypoints using MP Holistic
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_holistic = mp.solutions.holistic # Holistic model


"""

helper function definitions

"""

# fucntion to return mediapipe predictions
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)   # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False    #image no longer writeable to save computation space
    results = model.process(image)   # Make prediction
    image.flags.writeable = True    #image now writeable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # COLOR CONVERSION RGB 2 BGR
    return image,results




# Function to return arithmetic moving average of angles if more than 50% of angles are correct and less then 140
def quality_check(anglelist):
    
    addition = 0
    for angle in anglelist:
        if angle[0]<=140 and angle[1]==True:
            addition = addition + 1
    if(addition>=len(anglelist)/2):
        return moving_average(anglelist)
    else:
        return None

    
 

# Function that caclculates arithmetic moving average
def moving_average(anglelist):
    
    itera=2
    summ=list(anglelist)[0][0]
    for i in range(4,len(anglelist),5):
        summ = summ + itera*list(anglelist)[i][0]
        itera+=1
            
    return summ/( itera*(itera - 1)/2)


# function to draw and mark 3 landmarks of left/right leg
def draw_landmarks (frame,results,size,x,y,z):
    
    # draw landmarks
    mp_drawing.draw_landmarks(frame,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                      mp_drawing.DrawingSpec(color=(80,22,10), thickness = 1, circle_radius=3),
                      mp_drawing.DrawingSpec(color=(80,44,121), thickness = 1, circle_radius = 1))
    # mark landmarks    
    cv2.putText(frame,str(np.rint(np.multiply(y,size))*.1),tuple(np.multiply(y,size).astype(int)),
        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    cv2.putText(frame,str(np.rint(np.multiply(x,size))*.1),tuple(np.multiply(x,size).astype(int)),
        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    cv2.putText(frame,str(np.rint(np.multiply(z,size))*.1),tuple(np.multiply(z,size).astype(int)),
        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)

    
# checking if posture is correct usign geometry
def correct_posture(x,y):
    length = (np.cos(1.221730))*np.sqrt(np.sum(np.square(np.subtract(x,y))))*1000
    a=np.absolute(x[1]-y[1])*1000
    if (a>=length):
        return True,length,a
    else :
        return False,length,a
    

# Function to calculate
def cal_angle(frame,holistic,lowermost_landmark_of_patient,size):
    
    #initializatins
    angle = None
    correct,threshold,Actual = None,None,None
    
    #make detections
    image = frame.copy()  # creating a copy of frame
    image[int(lowermost_landmark_of_patient*size[1])+70:,:,:] = [0,0,0]  # mapping the frame portion below hip of person black to foster prediction
    image,results = mediapipe_detection(image,holistic)  

    #Getting the landmark coordinates and calculating angle 
    try:
        landmarks = results.pose_landmarks.landmark
        if landmarks[24].z > landmarks[23].z:  # conditon to check patien is sittle left to right or vice-versa
            key = [23,25,27]  # landarks of left leg
        else :
            key = [24,26,28]  # landarks of right leg
        try:
            a,b,c = key
            x=[landmarks[a].x,landmarks[a].y]
            y=[landmarks[b].x,landmarks[b].y]
            z=[landmarks[c].x,landmarks[c].y]
            
            #calculate angle using geometry
            radians = np.arctan2(x[1]-y[1], z[0]-y[0]) - np.arctan2(x[1]-y[1], x[0]-y[0])
            angle = np.abs(radians*180.0/np.pi)
            if angle >180:
                angle = 360 - angle 
            
            #drawing landmarks on frame
            draw_landmarks (frame,results,size,x,y,z)
            
            #checking if posture is correct or not
            correct,threshold,Actual = correct_posture(x,y)

        except:
            pass
    except:
        pass
    
    if angle is not None:
        return angle,correct,z[1],frame
    else :
        return None,correct,0.9,frame
    
    

    
"""

Main Function


"""

# variable initialization
angle_buffer = []  # temporary variable to store the last non-None value of angle 
laps = 0  #counter to record number of laps
anglelist = []  #list to keep track of last 'frame_threshold' number of angles
frame_threshold = 45  #Number of angles to keep track of for smoothening frame transition
ftime = 0  # counter to keep record of total time elapsed
stime = 0   # counter to keep record of time elapsed since the start of a lap
lowermost_landmark_of_patient = 0.9  # variable to keep track of 'hip' lankmark of pateint predicited


# Open the camera
cap = cv2.VideoCapture('KneeBendVideo.mp4')


# error handling and frame size and fps deduction
if (cap.isOpened() == False): 
    print("Error reading video file")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(vid_capture.get(5))
size = (frame_width, frame_height)


# Initialize video writer object
result = cv2.VideoWriter('trial_run.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         fps, size)


with mp_holistic.Holistic(min_detection_confidence = 0.5,min_tracking_confidence=0.5, model_complexity=1) as holistic:
    
    while (cap.isOpened()):
        
        ret, image = cap.read()
        ftime = ftime + 1/fps
        
        # common error handling with video streaming
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break    
        if ret==False:
            print('streaml ended!!')
            break
        
        # retrieving the angle made by knee,lowermost_landmark_of_patient and modified image
        angle0,angle1,lowermost_landmark_of_patient,image =cal_angle(image,holistic,lowermost_landmark_of_patient,size)
        angle = [angle0,angle1]
        if angle[0] is None: # storing previos angle if current angle is none
            angle = angle_buffer
        angle_buffer = angle

        
        # maintaining the anglelist's frame_count
        if len(anglelist)>=frame_threshold:
            iter = anglelist.pop(0)
        anglelist.append(angle)
       
        counter = 0 # counter to keep track of ongoing lap
        
        #information display on video frame
        cv2.rectangle(image,(0,0),(225,125),(255,255,255),-1)
        cv2.putText(image,'Session Time : '+str(int(ftime)),(5,15),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,'Laps completed : '+str(laps),(5,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,'Knee Angle : '+str(int(angle[0])),(5,45),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.rectangle(image,(5,65),(105,90),(0, 0, 0),1)
        
        
        #display information based on the conditions
        avgangle = quality_check(anglelist)
        if avgangle is not None and avgangle<=140:

            stime = stime + 1/30
            prob = float(stime)/8
            if prob<=1:
                cv2.rectangle(image,(7,67),(int(prob*100)+7,88),(11, 230, 51),-1)
                cv2.putText(image,str("Hold Position..."),(5,120),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            else:
                cv2.rectangle(image,(7,67),(int(prob*100)+7,88),(0, 0, 250),-1)
                cv2.putText(image,str("Straighten your leg..."),(5,120),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,'AMA Knee Angle : '+str(int(avgangle)),(5,60),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,'lap time : '+ str(int(stime)),(5,105),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            result.write(image)
            cv2.imshow('img',image)
            
        else:
            
            cv2.putText(image,str("Bend your knee correctly"),(5,105),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str("minimum for 8 secs"),(5,120),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            result.write(image)
            cv2.imshow('img',image)
            
            if stime>8: # set counter = 1 if one_lap time > 8 sec
                counter = 1
            stime = 1.5  # set intial lap_time = 1.5 seconds to account for the time taken by smoothening 
            
            
        if counter == 1:  #increasing lap count if counter is equal to 1
            laps = laps + 1
            
    cap.release()
    result.release()
    cv2.destroyAllWindows()

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates